In [1]:
import xlwings as xw
import pyodbc 
import pandas as pd
import numpy as np
import re
import os
import math
    

In [2]:
def get_dist_and_simi(path, sexRow = 15, ageRow = 4, cityRow = 20,intrestRow = 36, intrestNum = 56):
    allSheetName = pd.ExcelFile(path).sheet_names
    numSheet = len(allSheetName)
    
# get the distance: we use the Euclidean distance.
    def dist(a,b):
        dist = math.sqrt(sum((a-b)**2))
        return dist

    def simi(a,b):
        simi = 1/(1+math.sqrt(sum((a-b)**2)))
        return simi
    
# compile the data (Sex)
    profileSex = pd.DataFrame()
    for i in range(numSheet):
        profileSex[allSheetName[i]] = pd.read_excel(path,sheet_name=allSheetName[i]).iloc[sexRow-2:sexRow,1]/100
    
# Age
    profileAge = pd.DataFrame()
    for i in range(numSheet):
        profileAge[allSheetName[i]] = pd.read_excel(path,sheet_name=allSheetName[i]).iloc[ageRow-2:ageRow+5,1]/100
    
# City
    profileCity = pd.DataFrame()
    for i in range(numSheet):
        profileCity[allSheetName[i]] = pd.read_excel(path,sheet_name=allSheetName[i]).iloc[cityRow-2:cityRow+2,1]/100
# Interest
   
    '''兴趣数据（原数据按顺序排列）
    profileInterest = pd.DataFrame()
    for i in range(numSheet): 
        Interest = []
        for n in range(24,74):    

                if pd.read_excel(path,sheet_name=allSheetName[i]).iloc[n,0] in ('时事','娱乐明星','时尚','搞笑幽默','美女帅哥',
                                                                               '音乐','情感两性','电影','美食','美妆','教育','动漫'
                                                                               ,'读书作家','综艺节目','体育','摄影拍照','财经','游戏'
                                                                               ,'电视剧','母婴育儿','数码','旅游出行'):
                    Interest.append(pd.read_excel(path,sheet_name=allSheetName[i]).iloc[n,1]/100)
        profileInterest[allSheetName[i]] = Interest'''
    
# Primary Difference
    intrestList = ['电视剧','综艺','娱乐明星','情感','IT产业','日常生活','音乐','宠物','电影','美食',
                   '美妆','教育','动漫','社会','医疗','运动健身','读书作家','财经','游戏','体育','数码','旅游出行']
    profileInterestRaw = pd.DataFrame(index = intrestList)
    for i in range(numSheet):     
        Interest = []
        for n in range(22):
            for m in range(intrestRow-2,intrestRow+intrestNum-2):
                if pd.read_excel(path,sheet_name=allSheetName[i]).iloc[m,0] == profileInterestRaw.index[n]:
                    Interest.append(pd.read_excel(path,sheet_name=allSheetName[i]).iloc[m,1]/100)
        profileInterestRaw[allSheetName[i]] = Interest
# Standardization

    profileInterest = pd.DataFrame(index = intrestList, columns = allSheetName)
    for i in range(numSheet):   
        for n in range(22):
            profileInterest.iloc[n,i] = (profileInterestRaw.iloc[n,i]- min(profileInterestRaw.iloc[:,i]))/(max(profileInterestRaw.iloc[:,i]) - min(profileInterestRaw.iloc[:,i]))  
    
# merge data
    l1 = [profileSex, profileAge, profileCity]
    profileUser = pd.concat(l1)
    
    l2 = [profileUser, profileInterest]
    profileAll = pd.concat(l2)
    
# Distance Matrix
    distMatrix_user = pd.DataFrame()
    for i in range(numSheet):
        distance = []
        for n in range(numSheet):
            distance.append(dist(profileUser.iloc[:,i],profileUser.iloc[:,n]))

        distMatrix_user[allSheetName[i]] = distance

    distMatrix_user.index = distMatrix_user.columns
    
# simliarity Matrix
    simiMatrix_user = pd.DataFrame()
    for i in range(numSheet):
        similar = []
        for n in range(numSheet):
            similar.append(simi(profileUser.iloc[:,i],profileUser.iloc[:,n]))

        simiMatrix_user[allSheetName[i]] = similar

    simiMatrix_user.index = simiMatrix_user.columns
# Interest distance Matrix
    distMatrix_intrest = pd.DataFrame()
    for i in range(numSheet):
        distance = []
        for n in range(numSheet):
            distance.append(dist(profileInterest.iloc[:,i],profileInterest.iloc[:,n]))

        distMatrix_intrest[allSheetName[i]] = distance

    distMatrix_intrest.index = distMatrix_intrest.columns
    
# Interest similarity Matrix
    simiMatrix_intrest = pd.DataFrame()
    for i in range(numSheet):
        similar = []
        for n in range(numSheet):
            similar.append(simi(profileInterest.iloc[:,i],profileInterest.iloc[:,n]))

        simiMatrix_intrest[allSheetName[i]] = similar

    simiMatrix_intrest.index = simiMatrix_intrest.columns
    
# Total Matrix
    distMatrix_all = pd.DataFrame()
    for i in range(numSheet):
        distance = []
        for n in range(numSheet):
            distance.append(dist(profileAll.iloc[:,i],profileAll.iloc[:,n]))

        distMatrix_all[allSheetName[i]] = distance

    distMatrix_all.index = distMatrix_all.columns
    
# Total similarity matrix
    simiMatrix_all = pd.DataFrame()
    for i in range(numSheet):
        similar = []
        for n in range(numSheet):
            similar.append(simi(profileAll.iloc[:,i],profileAll.iloc[:,n]))

        simiMatrix_all[allSheetName[i]] = similar

    simiMatrix_all.index = simiMatrix_all.columns
    
    return distMatrix_user, simiMatrix_user, distMatrix_intrest, simiMatrix_intrest, distMatrix_all, simiMatrix_all

In [5]:
matrix = get_dist_and_simi(path = r'/Users/zihanzhao/Documents/Job Application/微博实习/人群画像相似度分析/波峰-波谷.xlsx',intrestNum = 56)
matrix = [m.rename(index={'波峰': 'peaks', '波谷': 'lowest'}, columns={'波峰': 'peaks', '波谷': 'lowest'}) for m in matrix]
matrix


[           peaks    lowest
 peaks   0.000000  0.057635
 lowest  0.057635  0.000000,
            peaks    lowest
 peaks   1.000000  0.945506
 lowest  0.945506  1.000000,
            peaks    lowest
 peaks   0.000000  0.471609
 lowest  0.471609  0.000000,
            peaks    lowest
 peaks   1.000000  0.679528
 lowest  0.679528  1.000000,
            peaks    lowest
 peaks   0.000000  0.475118
 lowest  0.475118  0.000000,
            peaks    lowest
 peaks   1.000000  0.677912
 lowest  0.677912  1.000000]

In [6]:
# writer = pd.ExcelWriter('波峰-波谷output.xlsx')
# total_similarity 
dimension_difference= matrix[0]
dimension_similarity = matrix[1]
interest_difference = matrix[2]
interest_similarity = matrix[3]
total_difference = matrix[4]
total_similarity  = matrix[5]
# writer.save()
# writer.close()
